# 🧪 Custom Bayes Search

In this notebook, we implement **Bayes Search with cross-validation** from scratch using **a custom class `MyBayesSearchCV`**. We then compare the performance of these implementations with **scikit-learn**'s `BayesSearchCV`, **optuna**'s  and **hyperopt**'s models.

### ⚙️ Importing Libraries & Environment Setup

In [91]:
from typing import Any

import numpy as np
import optuna
import pandas as pd
from numpy.typing import NDArray
from optuna.trial import Trial
from scipy.stats import gaussian_kde
from sklearn.base import BaseEstimator, clone
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
from sklearn.model_selection import (
    BaseCrossValidator,
    KFold,
    StratifiedKFold,
    cross_val_score,
    train_test_split,
)
from sklearn.svm import SVC
from skopt import BayesSearchCV
from skopt.space import Integer, Real

In [92]:
%matplotlib inline

pd.set_option("display.width", 150)
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)

### 📥 Loading the Dataset

In [93]:
# Load iris dataset
X, y = load_iris(return_X_y=True)

In [94]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

### 🧠 Implementing Custom Model Algorithms

In [95]:
class MyBayesSearchCV:
    """Bayesian hyperparameter optimization with cross-validation.

    This class combines random sampling with Bayesian optimization using a
    Tree-structured Parzen Estimator (TPE) approach to efficiently search
    for the best hyperparameters of an estimator. The search is guided by
    Expected Improvement, focusing on maximizing cross-validated accuracy.

    Attributes:
        estimator (BaseEstimator): The machine learning estimator to optimize.
        search_spaces (dict[str, Real | Integer]): Hyperparameter search distributions.
        n_iter (int): Total number of hyperparameter configurations to evaluate.
        cv (int | BaseCrossValidator): Cross-validation splitting strategy.
        random_state (int | None): Seed for reproducible randomness.
        best_estimator_ (BaseEstimator | None): Fitted estimator with best found params.
        best_params_ (dict[str, Any] | None): Best hyperparameters found during search.
        best_score_ (float): Best cross-validation accuracy score obtained.
    """

    def __init__(
        self,
        estimator: BaseEstimator,
        search_spaces: dict[str, Real | Integer],
        n_iter: int,
        cv: int | BaseCrossValidator = 5,
        random_state: int | None = None,
    ) -> None:
        """Initialize the Bayesian hyperparameter search.

        Args:
            estimator (BaseEstimator): Estimator object to optimize.
            search_spaces (dict[str, Real | Integer]): Dictionary mapping hyperparameter
                names to their search distributions (e.g. from scipy.stats).
            n_iter (int): Total number of parameter settings to sample.
            cv (int | BaseCrossValidator, optional): Cross-validation strategy,
                either an integer (number of folds) or a cross-validator instance.
                Defaults to 5.
            random_state (int | None, optional): Random seed for reproducibility.
                Defaults to None.
        """
        self.estimator = estimator
        self.search_spaces = search_spaces
        self.n_iter = n_iter
        self.n_candidates: int = 30
        self.gamma: float = 0.2

        if isinstance(cv, int):
            self.cv = KFold(n_splits=cv, shuffle=True, random_state=42)
        else:
            self.cv = cv

        self._rng = np.random.RandomState(random_state)
        self.best_estimator_: BaseEstimator | None = None
        self.best_params_: dict[str, Any] | None = None
        self.best_score_: float = float("-inf")
        self._X_: list[list[float]] = []
        self._y_: list[float] = []

        self._dimensions = list(self.search_spaces.values())
        self._param_names = list(self.search_spaces.keys())

    def _sample_params(self) -> tuple[dict[str, Any], list[float]]:
        """Sample a set of hyperparameters from the search space.

        Returns:
            tuple[dict[str, Any], list[float]]: A dictionary of sampled parameters and
                the corresponding parameter values as a list.
        """
        sampled = [dim.rvs(random_state=self._rng)[0] for dim in self._dimensions]
        params = dict(zip(self._param_names, sampled, strict=False))
        return params, sampled

    def _evaluate(
        self, params: dict[str, Any], X: NDArray[np.float64], y: NDArray[np.int64]
    ) -> float:
        """Evaluate model performance with given parameters using cross-validation.

        Args:
            params (dict[str, Any]): Parameters to evaluate.
            X (NDArray[np.float64]): Feature matrix.
            y (NDArray[np.int64]): Target labels.

        Returns:
            float: Mean accuracy across all CV folds.
        """
        estimator = clone(self.estimator)
        estimator.set_params(**params)

        scores = []
        for train_idx, valid_idx in self.cv.split(X, y):
            X_train, X_test = X[train_idx], X[valid_idx]
            y_train, y_test = y[train_idx], y[valid_idx]

            estimator.fit(X_train, y_train)
            y_pred = estimator.predict(X_test)

            score = accuracy_score(y_test, y_pred)
            scores.append(score)

        return float(np.mean(scores))

    def _safe_gaussian_kde(self, data: NDArray[np.float64]) -> callable:
        """Safely create a Gaussian KDE or fallback to uniform if data is degenerate.

        Args:
            data (NDArray[np.float64]): 1D array of data points.

        Returns:
            callable: KDE function or constant function returning 1.
        """
        if data.size < 2 or np.all(data == data[0]):
            return lambda x: [1.0]
        else:
            return gaussian_kde(data)

    def _tpe_suggest(
        self, X_sampled: NDArray[np.float64], y_sampled: NDArray[np.float64]
    ) -> tuple[dict[str, Any], list[float]]:
        """Suggest the next hyperparameter set to evaluate using TPE.

        The method fits KDEs to the best and the rest of the observed samples,
        then samples candidates and selects the one maximizing the ratio
        of densities (Expected Improvement).

        Args:
            X_sampled (NDArray[np.float64]): Previously sampled parameter vectors.
            y_sampled (NDArray[np.float64]): Corresponding scores for sampled params.

        Returns:
            tuple[dict[str, Any], list[float]]: Suggested hyperparameters.
        """
        n_best = max(2, int(self.gamma * len(y_sampled)))
        indices_sorted = np.argsort(y_sampled)[::-1]
        best_indices = indices_sorted[:n_best]
        rest_indices = indices_sorted[n_best:]

        X_best = X_sampled[best_indices]
        X_rest = X_sampled[rest_indices]

        kde_best = [
            self._safe_gaussian_kde(X_best[:, i]) for i in range(X_best.shape[1])
        ]
        kde_rest = [
            self._safe_gaussian_kde(X_rest[:, i]) for i in range(X_rest.shape[1])
        ]

        candidates_params = []
        candidates_x = []
        for _ in range(self.n_candidates):
            params, x = self._sample_params()
            candidates_params.append(params)
            candidates_x.append(x)

        scores = []
        for x in candidates_x:
            p_best = np.prod([kde_best[i](x[i])[0] for i in range(len(x))])
            p_rest = np.prod([kde_rest[i](x[i])[0] for i in range(len(x))])
            score = p_best / p_rest if p_rest > 0 else np.inf
            scores.append(score)

        best_idx = int(np.argmax(scores))
        return candidates_params[best_idx], candidates_x[best_idx]

    def fit(self, X: NDArray[np.float64], y: NDArray[np.int64 | np.float64]) -> None:
        """Perform hyperparameter search with Bayesian optimization.

        Args:
            X (NDArray[np.float64]): Feature matrix of shape (n_samples, n_features).
            y (NDArray[np.int64 | np.float64]): Target vector of shape (n_samples,).
        """
        init_points = min(10, self.n_iter)
        for _ in range(init_points):
            params, x = self._sample_params()
            score = self._evaluate(params, X, y)
            self._X_.append(x)
            self._y_.append(score)

            if self.best_score_ < score:
                self.best_score_ = score
                self.best_params_ = params

        for _ in range(self.n_iter - init_points):
            X_train = np.array(self._X_)
            y_train = np.array(self._y_)

            best_params, best_x = self._tpe_suggest(X_train, y_train)

            score = self._evaluate(best_params, X, y)
            self._X_.append(best_x)
            self._y_.append(score)

            if self.best_score_ < score:
                self.best_score_ = score
                self.best_params_ = best_params

        self.best_estimator_ = clone(self.estimator)
        self.best_estimator_.set_params(**self.best_params_)
        self.best_estimator_.fit(X, y)

    def predict(self, X: NDArray[np.float64]) -> NDArray[np.int64 | np.float64]:
        """Predict target values using the best found estimator.

        Args:
            X (NDArray[np.float64]): Feature matrix for prediction.

        Raises:
            ValueError: If fit() has not been called yet.

        Returns:
            NDArray[np.int64 | np.float64]: Predicted target values.
        """
        if self.best_estimator_ is None:
            raise ValueError("fit() must be called before predict()")
        return self.best_estimator_.predict(X)

    def score(self, X: NDArray[np.float64], y: NDArray[np.int64 | np.float64]) -> float:
        """Compute accuracy of the best estimator on the given data.

        Args:
            X (NDArray[np.float64]): Feature matrix.
            y (NDArray[np.int64]): True labels.

        Raises:
            ValueError: If fit() has not been called yet.

        Returns:
            float: Accuracy score.
        """
        if self.best_estimator_ is None:
            raise ValueError("fit() must be called before score()")
        return self.best_estimator_.score(X, y)

### 🏋️‍♂️ Model Training

In [96]:
# Base model
model = SVC(kernel="poly")

# Hyperparameter spaces
search_spaces = {
    "C": Real(1e-2, 100.0, prior="log-uniform"),
    "gamma": Real(1e-4, 1e-1, prior="log-uniform"),
    "degree": Integer(2, 5),
}


def objective(trial: Trial) -> float:
    """Objective function for Optuna study to optimize hyperparameters of SVC.

    Parameters:
        trial (Trial): A single trial object that suggests hyperparameter values.

    Returns:
        float: Cross-validated accuracy score (mean of folds).
    """
    C = trial.suggest_float("C", 1e-5, 100.0, log=True)
    gamma = trial.suggest_float("gamma", 1e-4, 1e-1, log=True)
    degree = trial.suggest_int("degree", 2, 5)

    model = SVC(C=C, gamma=gamma, degree=degree, kernel="poly")

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="accuracy")

    return np.mean(scores)


# Optuna
optuna_bs = optuna.create_study(direction="maximize")
optuna_bs.optimize(objective, n_trials=20, timeout=60)

# Scikit-learn BayesSearchCV
sklearn_bs = BayesSearchCV(
    estimator=model, search_spaces=search_spaces, n_iter=20, random_state=42
)
sklearn_bs.fit(X_train, y_train)

# My BayesSearchCV
my_bs = MyBayesSearchCV(
    estimator=model, search_spaces=search_spaces, n_iter=20, random_state=42
)
my_bs.fit(X_train, y_train)

[I 2025-07-30 19:50:47,875] A new study created in memory with name: no-name-bb0d4a86-dfce-43d8-ad5d-326815e0be8f
[I 2025-07-30 19:50:47,890] Trial 0 finished with value: 0.4333333333333333 and parameters: {'C': 1.091625102864551e-05, 'gamma': 0.00045798892646589477, 'degree': 4}. Best is trial 0 with value: 0.4333333333333333.
[I 2025-07-30 19:50:47,900] Trial 1 finished with value: 0.4749999999999999 and parameters: {'C': 0.03568114384119702, 'gamma': 0.005283939818976039, 'degree': 3}. Best is trial 1 with value: 0.4749999999999999.
[I 2025-07-30 19:50:47,911] Trial 2 finished with value: 0.4333333333333333 and parameters: {'C': 26.133107914438046, 'gamma': 0.002225465625781207, 'degree': 4}. Best is trial 1 with value: 0.4749999999999999.
[I 2025-07-30 19:50:47,922] Trial 3 finished with value: 0.9666666666666667 and parameters: {'C': 19.796941324709028, 'gamma': 0.015756113937268366, 'degree': 3}. Best is trial 3 with value: 0.9666666666666667.
[I 2025-07-30 19:50:47,933] Trial 4 

### 📊 Comparing Algorithm Versions

In [97]:
optuna_model = clone(model)
optuna_model.set_params(**optuna_bs.best_params)
optuna_model.fit(X_train, y_train)
optuna_pred = optuna_model.predict(X_test)
optuna_test_score = accuracy_score(optuna_pred, y_test)

In [98]:
bayes_search_algorithms = {
    "sklearn": {
        **sklearn_bs.best_params_,
        "best_cv_score": sklearn_bs.best_score_,
        "test_score": sklearn_bs.score(X_test, y_test),
    },
    "optuna": {
        **optuna_bs.best_params,
        "best_cv_score": optuna_bs.best_value,
        "test_score": optuna_test_score,
    },
    "custom": {
        **my_bs.best_params_,
        "best_cv_score": my_bs.best_score_,
        "test_score": my_bs.score(X_test, y_test),
    },
}

pd.DataFrame(bayes_search_algorithms).T

,C,degree,gamma,best_cv_score,test_score
sklearn,100.000000,2.0,0.015539,0.958333,1.0
optuna,3.696561,3.0,0.023089,0.975000,1.0
custom,0.029349,4.0,0.065993,0.966667,1.0


In [ ]:
def objective(trial: Trial) -> float:
    """Objective function for Optuna study to optimize hyperparameters.

    Parameters:
        trial (Trial): A single trial object that suggests hyperparameter values.

    Returns:
        float: Cross-validated accuracy score (mean of folds).
    """

    model_name = trial.suggest_categorical('model', ['xgb', 'rf', 'svm'])

    if model_name == 'xgb':
        params = {
            'n_estimators': trial.suggest_int('xgb_n_estimators', 50, 300),
            'max_depth': trial.suggest_int('xgb_max_depth', 3, 12),
            'learning_rate': trial.suggest_float('xgb_learning_rate', 0.01, 0.3, log=True),
            'subsample': trial.suggest_float('xgb_subsample', 0.5, 1.0),
            'colsample_bytree': trial.suggest_float('xgb_colsample_bytree', 0.5, 1.0),
            'use_label_encoder': False,
            'eval_metric': 'logloss',
            'random_state': 42
        }
        model = xgb.XGBClassifier(**params)

    elif model_name == 'svm':
        params = {
            'C': trial.suggest_float('svm_C', 1e-3, 100, log=True),
            'kernel': trial.suggest_categorical('svm_kernel', ['linear', 'rbf', 'poly']),
            'gamma': 'scale',
        'random_state': 42
        }
        if params['kernel'] == 'poly':
            params['degree'] = trial.suggest_int('svm_degree', 2, 5)
        model = SVC(**params)

    else:
        params = {
            'n_estimators': trial.suggest_int('rf_n_estimators', 50, 300),
            'max_depth': trial.suggest_int('rf_max_depth', 3, 30),
            'max_features': trial.suggest_categorical('rf_max_features', ['sqrt', 'log2', None]),
            'min_samples_split': trial.suggest_int('rf_min_samples_split', 2, 10),
            'random_state': 42,
            'n_jobs': -1
        }
        model = RandomForestClassifier(**params)

    cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    scores = cross_val_score(model, X_train, y_train, cv=cv, scoring="accuracy")

    return np.mean(scores)

estimators = {
    "rf": RandomForestClassifier,
    "xgb": XGBClassifier,
    "svm"; SVC,
}


optuna_bs = optuna.create_study(direction="maximize")
optuna_bs.optimize(objective, n_trials=20, timeout=60)

params = study.best_params
model_name = params.pop('model')

model = estimators[model_name](**params)
model.fit(X_train, y_train)
y_pred = optuna_model.predict(X_test)